# Microgrid Location Optimization using Gurobi
This notebook demonstrates how to solve the microgrid location optimization problem using classical optimization techniques with Gurobi. We'll compare the results with quantum optimization approaches and provide comprehensive visualizations.
    
## Table of Contents
1. [Setup and Data Loading](#setup)
2. [Microgrid Location Generation](#generation)
3. [Cost Matrix Construction](#cost-matrices)
4. [Gurobi Optimization](#optimization)
5. [Results Visualization](#visualization)
6. [Comparison with Quantum Solutions](#comparison)
7. [Sensitivity Analysis](#sensitivity)

## 1. Setup and Data Loading

In [ ]:
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.distance import geodesic
import gurobipy as gp
from gurobipy import GRB
import warnings
warnings.filterwarnings('ignore')
    
# Set style for better plots
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
print("Libraries imported successfully!")

# Load the dataset from CSV
df = pd.read_csv('/Users/haikelhaikel/QuantathonQR/QRenew/database/SolarPowerPlants_TH.csv', delimiter=',', skiprows=1)
df

In [ ]:
ind = df['Index']
name = df['name']
lat = df['latitude']
long = df['longitude']
capacity = df['capacity(MW)']
penetration = df['Total price']

## 2. Microgrid Location Generation

In [ ]:
# generate choices for microgrid locations
num_microgrids = 20 ## number of microgrids
random_lat = np.random.uniform(min(lat), max(lat), size=4*num_microgrids)
lat_choices = np.random.choice(random_lat, size=num_microgrids, replace=False)

random_long = np.random.uniform(min(long), max(long), size=4*num_microgrids)
long_choices = np.random.choice(random_long, size=num_microgrids, replace=False)
        
print(f"Generated {num_microgrids} potential microgrid locations")

## 3. Cost Matrix Construction

In [ ]:
def calculate_distance_mgpp(mglat, mglong, pplat, pplong):    
    distance_matrix = np.zeros(len(pplat))
    for i in range(len(pplat)):
        distance_matrix[i] = geodesic((mglat, mglong), (pplat[i], pplong[i])).km
    return distance_matrix
    
def calculate_distance_mgmg(mglat, mglong):    
    distance_matrix = np.zeros((len(mglat), len(mglat)))
    for i in range(len(mglat)):
        for j in range(len(mglat)):
            if i < j:
                distance_matrix[i][j] = geodesic((mglat[i], mglong[i]), (mglat[j], mglong[j])).km
    return distance_matrix

def build_cost_matrices(self, cost_coeff=10000, battery_cost=38000, solar_cost=15000):
        """Build cost matrices for the optimization problem"""
        if self.microgrid_locations is None:
            raise ValueError("Microgrid locations must be generated first")
            
        num_microgrids = len(self.microgrid_locations)
        
        # Calculate total distance from each microgrid to all power plants
        mgpp_distances = np.zeros(num_microgrids)
        for i, (lat, long) in enumerate(self.microgrid_locations):
            mgpp_distances[i] = np.sum(self.calculate_distance_mg_pp(lat, long))
        
        # Calculate distances between microgrids
        mgmg_distances = self.calculate_distance_mg_mg()
        
        # Convert distances to costs
        mgpp_costs = mgpp_distances * cost_coeff
        mgmg_costs = mgmg_distances * cost_coeff
        
        # Fixed costs for installation
        fixed_cost = battery_cost + solar_cost
        
        # Get penetration values (assuming these are benefits/revenues)
        penetration = self.solar_data['Total price'].values
        if len(penetration) < num_microgrids:
            # If we have fewer power plants than microgrids, repeat the pattern
            penetration = np.tile(penetration, (num_microgrids // len(penetration)) + 1)[:num_microgrids]
        elif len(penetration) > num_microgrids:
            penetration = penetration[:num_microgrids]
        
        return mgpp_costs, mgmg_costs, fixed_cost, penetration

## 4. Gurobi Optimization